In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from six.moves import cPickle as pickle
from six.moves import range

import time

In [3]:
pickle_file = 'SVHN_1x64x64_valid.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['valid_dataset']
    train_labels = save['valid_labels']
    del save  # hint to help gc free up memory
    print('train set', train_dataset.shape, train_labels.shape)

train set (5679, 1, 64, 64) (5679, 6)


In [4]:
c3 = []
c2 = []
c1 = []

for i in range(5679):
    categ = train_labels[i][0]
    if(categ == 3):
        c3.append(i)
    elif(categ == 2):
        c2.append(i)
    elif(categ == 1):
        c1.append(i)

print(len(c3))
print(len(c2))
print(len(c1))

1545
2988
985


In [5]:
c3_data = train_dataset[c3]
c3_target = train_labels[c3]
print(c3_data.shape)
print(c3_target.shape)

(1545, 1, 64, 64)
(1545, 6)


In [6]:
c2_data = train_dataset[c2]
c2_target = train_labels[c2]
print(c2_data.shape)
print(c2_target.shape)

(2988, 1, 64, 64)
(2988, 6)


In [7]:
c1_data = train_dataset[c1]
c1_target = train_labels[c1]
print(c1_data.shape)
print(c1_target.shape)

(985, 1, 64, 64)
(985, 6)


In [8]:
numpy_data = np.concatenate((c1_data, c2_data, c3_data))
numpy_target = np.concatenate((c1_target, c2_target, c3_target))
print(numpy_data.shape, numpy_target.shape )

(5518, 1, 64, 64) (5518, 6)


In [9]:
del c1_data, c2_data, c3_data

In [10]:
del c1_target, c2_target, c3_target

In [11]:
del train_dataset, train_labels

In [12]:
del c3, c2, c1

In [13]:
rng_state = np.random.get_state()
X = np.random.permutation(numpy_data)

In [14]:
np.random.set_state(rng_state)
Y = np.random.permutation(numpy_target)

In [15]:
rng_state = np.random.get_state()
X = np.random.permutation(X)

In [16]:
np.random.set_state(rng_state)
Y = np.random.permutation(Y)

In [17]:
import os

In [18]:
pickle_file = 'SVHN_digits_3_2_1_64x64x1_valid.pickle'
try:
    f = open(pickle_file, 'wb')
    save = {
        'valid_data': X,
        'valid_target': Y
            }
    pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)
    f.close()
except Exception as e:
    print('Unable to save data to', pickle_file, ':', e)
    raise
statinfo = os.stat(pickle_file)
print('Compressed pickle size:', statinfo.st_size)

Compressed pickle size: 90539623


In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 3, padding=(1, 1))
        self.conv2 = nn.Conv2d(20, 40, 3, padding=(1, 1))
        self.conv3 = nn.Conv2d(40, 80, 3, padding=(1, 1))
        self.conv4 = nn.Conv2d(80, 120, 3, padding=(1, 1))
        self.conv5 = nn.Conv2d(120, 160, 3, padding=(1, 1))
        self.conv6 = nn.Conv2d(160, 200, 3, padding=(1, 1))
        self.conv7 = nn.Conv2d(200, 240, 3, padding=(1, 1))
        self.pool = nn.MaxPool2d(2, 2)
        self.FC = nn.Linear(960, 1080)
        self.digitlength = nn.Linear(1080, 7)
        self.digit1 = nn.Linear(1080, 10)
        self.digit2 = nn.Linear(1080, 10)
        self.digit3 = nn.Linear(1080, 10)
        self.digit4 = nn.Linear(1080, 10)
        self.digit5 = nn.Linear(1080, 10)
        
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                init.kaiming_normal(m.weight)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                init.kaiming_normal(m.weight)
                m.bias.data.zero_()
    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(F.relu(self.conv2(x)))
        x = F.relu(self.conv3(x))
        x = self.pool(F.relu(self.conv4(x)))
        x = self.pool(F.relu(self.conv5(x)))
        x = self.pool(F.relu(self.conv6(x)))
        x = self.pool(F.relu(self.conv7(x)))
        x = x.view(-1, 960)
        x = self.FC(x)
        yl = self.digitlength(x)
        y1 = self.digit1(x)
        y2 = self.digit2(x)
        y3 = self.digit3(x)
        y4 = self.digit4(x)
        y5 = self.digit5(x)
        return [yl, y1, y2, y3, y4, y5]

In [ ]:
net = Net()
net.cuda()

In [ ]:
for param in net.parameters():
    if(param.grad is not None):
        print(param)

In [ ]:
c3_data_tensor = torch.from_numpy(c3_data)
c3_target_tensor = torch.from_numpy(c3_target).type(torch.LongTensor)
print(c3_data_tensor.type(), c3_data_tensor.size())
print(c3_target_tensor.type(), c3_target_tensor.size())

In [ ]:
c2_data_tensor = torch.from_numpy(c2_data)
c2_target_tensor = torch.from_numpy(c2_target).type(torch.LongTensor)
print(c2_data_tensor.type(), c2_data_tensor.size())
print(c2_target_tensor.type(), c2_target_tensor.size())

In [ ]:
c1_data_tensor = torch.from_numpy(c1_data)
c1_target_tensor = torch.from_numpy(c1_target).type(torch.LongTensor)
print(c1_data_tensor.type(), c1_data_tensor.size())
print(c1_target_tensor.type(), c1_target_tensor.size())

In [ ]:
data_tensor = torch.cat((c3_data_tensor, c2_data_tensor, c1_data_tensor), 0)
target_tensor = torch.cat((c3_target_tensor, c2_target_tensor, c1_target_tensor), 0)
print(data_tensor.type(), data_tensor.size())
print(target_tensor.type(), target_tensor.size())

In [ ]:
objective = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters())

In [ ]:
num_epochs = 20
batch_size = 64
num_train =  c3_data.shape[0]
iter_per_epoch = num_train // batch_size
print_every = 300
print(iter_per_epoch)

In [ ]:
epoch_losses = {i:[] for i in range(num_epochs)}
loss_history = []

In [ ]:
a = torch.randn(10, 5)
print(a)

In [ ]:
idx = torch.randperm(5)
print(idx)

In [ ]:
b = a[idx]
print(b)

In [ ]:
print(id(a))
print(id(b))

In [ ]:
b = Variable(b).cuda()

In [ ]:
print(id(b))

In [ ]:
del b

In [ ]:
print(a)

In [ ]:
for epoch in range(num_epochs):  # loop over the dataset multiple times
    
    print('Epoch {}/{}'.format(epoch, num_epochs - 1))
    print('-' * 110)

    i = 0
    rng_state = torch.get_rng_state()
    new_idxs = torch.randperm(num_train)
    
    t1 = time.time()
    for t in range(iter_per_epoch):
        batch_idxs = new_idxs[i: i+batch_size]
        X_batch = data_tensor[batch_idxs]
        Y_batch = target_tensor[batch_idxs][:,0:4]
        i += batch_size

        X_batch = Variable(X_batch).cuda()
        optimizer.zero_grad()
        outputs = net(X_batch)
        
        if 
        
        lossl = objective(outputs[0], Y_batch[:, 0])
        loss1 = objective(outputs[1], Y_batch[:, 1])
        loss2 = objective(outputs[2], Y_batch[:, 2])
        loss3 = objective(outputs[3], Y_batch[:, 3])
        final_loss = lossl + loss1 + loss2 + loss3
        
        final_loss.backward()
        
        optimizer.step()
        
        loss_history.append(final_loss.data[0])
        epoch_losses[epoch].append(final_loss.data[0])
        
        if (t % print_every == 0):
            print('Iteration : ', t+1, ' / ', iter_per_epoch)
            print('loss : ', final_loss.data[0])
            print('lossl : ', lossl.data[0], 'loss1 : ', loss1.data[0], 'loss2 : ', loss2.data[0], 'loss3 : ', loss3.data[0])
        
    t2 = time.time()
    print("time taken : ", t2-t1)
    print('-' * 110)
        

In [ ]:
arrayidx = torch.randperm(5)
print(arrayidx)

In [ ]:
print(a[arrayidx])

In [ ]:
f = open("c3_leg_ep_20.pkl", "bw")
torch.save(net.state_dict(), f)
f.close()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.figure()
plt.plot(loss_history)